In [213]:
import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pdf2image import convert_from_path
import os
import pytesseract
from PIL import Image
import shutil
import cv2
import decimal
from zipfile import ZipFile
import glob
import tabula

In [16]:
zippath = '/Users/deniszagorodnev/Desktop/260.zip'

In [113]:
#распаковка файлов из скаченного архива

def unzipper(zippath, cond = False):
    
    dirpath = zippath[:-4] + '_package'
    
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
        
    os.mkdir(dirpath)

    with ZipFile(zippath, 'r') as zipObj:
        
        if cond:        
            listOfFileNames = zipObj.namelist()
            for fileName in listOfFileNames:
                if fileName.endswith('.csv'):
                    zipObj.extract(fileName, 'temp_csv')
        else:
            zipObj.extractall(path = dirpath)

In [127]:
#unzipper(zippath)

In [121]:
#выгрузка картинок из файла с аудиторским заключением (используем для самого свежего)

def audit_recog(fname):
    
    dirpath = '/Users/deniszagorodnev/Desktop/260_audit_imgs/'

    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)

    os.mkdir(dirpath)
    
    
        
    pdfFileObj = open(fname,'rb')

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    num_pages = pdfReader.numPages

    pages = convert_from_path(fname, 100)

    for i in range(len(pages)):
            page = pages[i]
            pname = dirpath + str(i) + '.jpg'
            page.save(pname, 'JPEG')

In [218]:
#обработка всех документов из распакованного архива 

def docs_recog(package_dir):

    

#обработка файлов с проектными декларациями
    print(glob.glob(package_dir + '/*_pd*.pdf'))

In [219]:
package_dir = '/Users/deniszagorodnev/Desktop'

docs_recog(package_dir)

['/Users/deniszagorodnev/Desktop/dev260_pd77-000555_01.pdf']


In [208]:
declar = '/Users/deniszagorodnev/Desktop/260_package/dev260_pd77-000554_01.pdf'
output = "/Users/deniszagorodnev/Desktop/declar.csv"

In [209]:
df = tabula.read_pdf(declar, pages = 'all', lattice = True)[1]

AttributeError: module 'tabula' has no attribute 'read_pdf'

In [159]:

df = tabula.read_pdf_table(declar, pages='all')
tabula.convert_into(df, output , output_format="csv",pages='all', stream=True)

AttributeError: module 'tabula' has no attribute 'read_pdf_table'

In [158]:
#обработка текста с всех картинок в файле

def ocr_core(folder):
    
    dirpath = folder + '_text_recog'
    
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
        
    os.mkdir(dirpath)
    
    for elem in os.listdir(folder):
        if elem.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = pytesseract.image_to_string(Image.open(folder + '/' + elem), lang="rus") 
            text_file = open(dirpath + '/' + str(elem[:-4]), "w")
            text_file.write(text)
            text_file.close()
    



In [204]:
declar = '/Users/deniszagorodnev/Desktop/260_package/dev260_fin_calc2017-4qr_01.pdf'

pdfFileObj = open(declar,'rb')

pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

num_pages = pdfReader.numPages
count = 0
text = ""

while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()

if text != "":
    text = text
else:
    text = textract.process(declar, method='tesseract', language='rus')

In [205]:
text_file = open('/Users/deniszagorodnev/Desktop/res', "wb")
text_file.write(text)
text_file.close()

In [126]:
#декодирование текста из штрихкода аудиторского заключения
def decoder(string):

    res = ''.join(chr(ord('а') + ord(sym) - ord('\\')) for sym in string)
    space = 'ϴ'
    
    for i in range(len(res)):
        if res[i] == space:
            res = res[:i] + ' ' + res[i+1:] 
        elif res[i] == 'ё':
            res = res[:i] + 'д' + res[i+1:] 
    return res       
        